## Imports

In [ ]:
import os
import numpy as np

import torch
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import CIFAR10, MNIST

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Model Settings

In [ ]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 28*28

# Other
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
GRAYSCALE = False

### CIFAR10 Dataset

In [ ]:
transform = (
    transforms.Compose([
        #transforms.Resize(224),
        #transforms.RandomHorizontalFlip(p=.40),
        #transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465), 
            (0.2471, 0.2435, 0.2616)
        )
    ])
)

In [ ]:
##########################
### CIFAR10 DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = CIFAR10(root='data', 
                        train=True, 
                        transform=transform, 
                        download=True)

test_dataset = CIFAR10(root='data', 
                       train=False, 
                       transform=transform)


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print(DEVICE)
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
cuda
Image batch dimensions: torch.Size([128, 3, 32, 32])
Image label dimensions: torch.Size([128])


In [ ]:
device = torch.device(DEVICE)
torch.manual_seed(0)

for epoch in range(2):
    for batch_idx, (x, y) in enumerate(train_loader):
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

Epoch: 1 | Batch index: 0 | Batch size: 128
Epoch: 2 | Batch index: 0 | Batch size: 128


## Model

In [ ]:
from google.colab import drive


drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys


#PATH = "/content/drive/My Drive/Colab Notebooks/EECS553ML_reproduce/"
PATH = "/content/drive/My Drive/EECS553ML_reproduce/"
sys.path.append(PATH)

In [ ]:
!cd "/content/drive/My Drive/EECS553ML_reproduce/";ls

 cifar10_resnet152_model.pt	'Model complexities.docx'   resnet18.ipynb
 cifar10_resnet152_train.ipynb	 __pycache__		    resnet18.py
 dataset.py			 PyTorch_CIFAR10	    ResNet-34
 mnist_resnet152_model.pt	 report.pdf		    resnet34.bak
 mnist_resnet152_train.ipynb	 ResNet-152		    resnet34.ipynb
 mnist_resnet18_model.t7	 resnet152.ipynb	    resnet34.py
 mnist_resnet34_model.t7	 resnet152.py		    results.png
 mnist_resnet34_train.ipynb	 ResNet-18		    trainer.py
 mnist_test.ipynb		 resnet18.bak


In [ ]:
from resnet18 import ResNet152


torch.manual_seed(RANDOM_SEED)
model = ResNet152()

## Training

In [ ]:
from trainer import Trainer


trainer = Trainer(model, train_loader, DEVICE)

Training on cuda


In [ ]:
trainer.train(LEARNING_RATE, NUM_EPOCHS)

Epoch: 001/010 | Batch 0000/0391 | Cost: 2.4060
Epoch: 001/010 | Batch 0050/0391 | Cost: 2.1325
Epoch: 001/010 | Batch 0100/0391 | Cost: 1.8973
Epoch: 001/010 | Batch 0150/0391 | Cost: 1.5841
Epoch: 001/010 | Batch 0200/0391 | Cost: 1.4192
Epoch: 001/010 | Batch 0250/0391 | Cost: 1.4456
Epoch: 001/010 | Batch 0300/0391 | Cost: 1.4382
Epoch: 001/010 | Batch 0350/0391 | Cost: 1.3460
Epoch: 001/010 | Train: 48.236%
Time elapsed: 10.61 min
Epoch: 002/010 | Batch 0000/0391 | Cost: 1.3695
Epoch: 002/010 | Batch 0050/0391 | Cost: 1.1349
Epoch: 002/010 | Batch 0100/0391 | Cost: 1.2547
Epoch: 002/010 | Batch 0150/0391 | Cost: 1.1938
Epoch: 002/010 | Batch 0200/0391 | Cost: 1.1821
Epoch: 002/010 | Batch 0250/0391 | Cost: 0.9206
Epoch: 002/010 | Batch 0300/0391 | Cost: 0.9768
Epoch: 002/010 | Batch 0350/0391 | Cost: 0.9028
Epoch: 002/010 | Train: 68.498%
Time elapsed: 21.10 min
Epoch: 003/010 | Batch 0000/0391 | Cost: 0.8270
Epoch: 003/010 | Batch 0050/0391 | Cost: 0.9235
Epoch: 003/010 | Batch 0

## Evaluation

In [ ]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (trainer.compute_accuracy(test_loader)))

Test accuracy: 81.65%


In [ ]:
#%watermark -iv

In [ ]:
torch.save(model.state_dict(), f"{PATH}cifar10_resnet152_model.t7")